## Starting Point

Neural Networks are excellent function approximators, so in a data rich world such as finance it would seem possible to train neural networks to learn these option pricing function. The question is **why** would we do this, when we have perfectly good and well established computer models that can already do it.

## Terminology
* **Option** a financial contract with a value contingent on some initial terms and how the financial markets move during the life of the contract.
* **Model** a mathematical algorithm that can determine the value of a specific type of option.
* **Neural Network** or **NN** A function approximator based on matrix manipulation that can be trained to find an approximate general form of a function given sufficient examples of input parameters and corresponding result(s).

## Motivation

There are many simple options that can be modelled *closed form*, i.e. with an equation, where we pass the valuation details and with relatively little compute cost and delay we are given a reliable valuation. 

However there are more complex options that have no such *closed form* as they have complex features. Typically these are *path dependent*, as in it matters how their valuation parameter change over time, and to solve these a very common solution is to run many simulations of how the valuation parameters will evolve in the future. We then valuation the option on all of these simulated future paths. It is then possible to combine the results of the simulations and establish the optimal price of the option. Where the *optimal* price is the price that would compensate the seller for entering into a contract where there is a likelihood they will have to pay out to tye buyer.

In these path dependent case, the number of simulations needed to get an accurate price can be very high. As such the compute costs is high and the time to run the calculation is also long, relative to the *closed form* cases.

Typically when selling (offering) options, the seller will make a market, by pricing many different terms and will also respond to **r**equest **f**or **q**uotes (RFQs) during the trading day. This can amount to a very significant number of calculations and as such significant compute costs eating into margins. In addition the time it can take to quite may be seen as client differentiating if it can be done more quickly than other sellers. In addition once a trade is done, the seller must re-value the trade as the market moves so they can understand their profit and loss as well as [hedge](https://www.investopedia.com/terms/h/hedge.asp) their risks. Again, this can involve a significant number of costly calculations.

## Hypothesis  

A neural network is slow to train but quick when making a prediction, as it is in essence a simple matrix multiplication. So the hypothesis we look at here is:
> Can a neural network be trained to estimate the financial option contract pay-offs and be accurate enough to have commercial use.
* If they can be trained is it more cost effective to train a neural network than to run it on demand as needed
* and is the neural network quicker when predicting than the equivalent direct model call.
* Then finally, are there viable commercial uses, proprietary trading, client valuations and approved in regulatory settings.

## Considerations

* Normal Day to Day operations generate many option valuations, which if captured this data could be used as training data
* Such, Day to Day data may not cover all possible scenarios, so might need to be augmented by creating specific training data
* Some complex financial options have many features, and feature spaces of 50 to 100 would require a significant amount of training data to ensure coverage in high such dimensions.
* Neural networks will always make a prediction, even when untrained.
* From a client and regulatory point of view [expandability](https://en.wikipedia.org/wiki/Explainable_artificial_intelligence) is a challenge as the seller may not be able to show why a certain price was given.
* As the markets move, the neural will need re-training, however this can be done by re-training the existing neural network with additional data, but it needs to be monitored.
* Accuracy will not be consistent, it will depend on where the prediction falls in feature space and how well covered that region is.
* Availability of data, the sellers organization will need to be set-up to value data and its monetization. Otherwise the cost of souring the data and changing the organization mindset will be prohibitive.

## Approach

Train a Neural Net to predict a:

1. Simple closed form model. This is not a target use case but is a first step in the proof.
2. Simple closed form model, with a simulated barrier condition to see how the NN behaves with step changes in price
3. Path dependent option with a small feature space.
4. Extrapolate the results to consider
   1. Data demands to train the neural network
   2. Relative cost of compute, on-demand vs trained neural network
   3. Relative speed of pricing 
   4. Pricing stability and accuracy
   5. Operational challenges and re-training
   6. Client concerns
   7. Seller concerns
   8. Regulatory concerns
   9. Go, No-Go on continuing the experiment with a high feature NN or 50+ dimensions in feature space.


In [ ]:
#
# General Python utilities and imports.
#
import inspect
from dataclasses import dataclass
from typing import Final
from typing import Tuple, List, Union, Callable
from functools import partial
from enum import Enum

In [ ]:
#
# Some set-up for the NN environment, for this we are using TensorFlow and Keras.
#
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib  # part of sklearn

tf.keras.backend.set_floatx('float64') # We want high accuracy for our regressions, slower then float-32, but necessary for this use case.

In [ ]:
#
# This is so we can visualize results
#
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable

%matplotlib inline

## Simple Closed Form Model

Our [closed form](https://en.wikipedia.org/wiki/Closed-form_expression) model is the [Black Scholes](https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model) option model. This will allow us to show the work flow and highlight the challenges with the approach.

The specific model implementation below is arbitrary, it is just a means to show how the neural network can learn a function of the same order of magnitude of complexity as a simple option contract.

The model also takes an function that inspects the price, volatility and Time to maturity and *optionally* modifies the price if it hit a condition in terms of price, volatility and time. This is to allow us to model the case where the option price has abrupt changes rather than smooth ones. Abrupt changes will be more of a challenge for the model to capture. We do this as many commercially traded options do have such [barrier](https://en.wikipedia.org/wiki/Barrier_option) features (knock-in, knock-out, up-and-out etc.)

This naive implementation will run on the CPU only. There are techniques to utilize all CPU cores, however these vary by language C++, Python etc and are non trivial to get right and distract from the problem at hand. As we will see later using tensorflow, it is possible to create language independent computation graphs that can fully utilize the GPU(s) available. This is s significant step up as a contemporary GPU will have approx. 10'000 cores compared to a CPU with approx 16 cores.

In [ ]:
@dataclass(frozen=True)
class BSConstants:
    call: Final[int] = 0
    put: Final[int] = 1
    price: Final[int] = 0
    delta: Final[int] = 1


def black_scholes_model(
    S: float,
    K: float,
    T: float,
    r: float,
    v: float,
    o: int = BSConstants.call,
    barrier: Callable[[float, float, float, float], float] = None,
    with_delta: bool = False,
) -> Tuple[float, float]:
    """
    Implementation of Black Scholes model.

    Trivial example to demo of proof of concept for fitting an option pricing model
    with a neural network.

    :param S: Spot
    :param K: Strike
    :param T: Time to maturity
    :param r: risk free rate
    :param v: underlying volatility
    :param o: type, call = 0 or put = 1
    :param barrier: A callable that takes: price, spot, volatility and time and returns a modified price if a certain vol and/or time barrier has been hit
    :param with_delta: If true calculate option delta, else return None for delta
    :return: Option Price, Delta
    """
    price: float = None
    delta: float = None

    d1 = (np.log(S / K) + (r + 0.5 * v**2) * T) / (v * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * v**2) * T) / (v * np.sqrt(T))

    if o == BSConstants.call:
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        if with_delta:
            delta = norm.cdf(d1)
    elif o == BSConstants.put:
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        if with_delta:
            delta = float(1) - norm.cdf(d1)
    else:
        assert False, "Option must be call=0 or put=1"

    if barrier is not None:
        return barrier(price, S, v, T)

    return (price, delta)

## Pricing Scenarios

In the real use case we will collect the back history  [RFQ (request for quote)](https://www.investopedia.com/terms/r/request-for-quote.asp) as training data for the neural network. This presents the first challenge as to train the model we need a spread of data across all combinations of all parameters and instrument details that impact the price we are trying to predict. If we have gaps in our training data there will also be a weakness in the model when predicting prices with that specific combination of inputs.

The nature of neural networks is to give results, even an totally untrained neural network will make predictions, just erroneous ones. So we will need to review the spread of inputs we have and make sure (expert judgment) that there are sufficient examples for all of the combinations we are likely to quote. If the markets shift significantly we can also add additional training data, but we must be aware when the model is being asked to make predictions outside of its experience.

In this example we will create training data from scratch by running scenarios by varying the input parameters and capturing the results.

**Note** 

* The Spot stock price is not that of any specific stock and the risk free rate is not that of a specific currency. Also, time is treated as a continuous variable, where zero = now. So the model is not linked to a specific stock, currency or time, so, in this simple case, we do not need to train our neural network for specific combinations of stock, currency & time as the NN will learn the general relationship as captured in the model.
* However, stock prices and risk free rates vary by stock and currency, so we must make sure the training examples capture the full range for these values for all stock and rates we will encounter. 
* Also all inputs are floating point numbers, which do not respect when stocks trade at integer values and where stocks and rates are precision limited to e.g. cents and basis points. So we must interpret the result accordingly with market conventions.

In [ ]:
@dataclass(frozen=True)
class MarketConstants:
    volMinStep: Final[float] = 0.01
    volMin: Final[float] = 0.01  # 1%
    volMax: Final[float] = 0.4  # 40%
    timeMinStep: Final[float] = (
        1.0 / 365.0
    )  # One day (technically not all days are trading days .. but)
    timeMin: Final[float] = 1e-9
    timeMax: Final[float] = 1.0  # One Year
    spotMinStep: Final[float] = 1.0 / 100.0
    spotMin: Final[float] = spotMinStep
    spotMax: Final[float] = 100.0
    strikeMinStep: Final[float] = spotMinStep
    strikeMin: Final[float] = spotMin
    strikeMax: Final[float] = spotMax
    rateMinStep: Final[float] = 1.0 / 10000.0  # one basis point
    rateMin: Final[float] = rateMinStep
    rateMax: Final[float] = 0.25  # 25%

In [ ]:
def stepped_range(
        min_value: float,
        max_value: float,
        num_steps: float,
        min_step_size: float = None
) -> List[float]:
    """

    Create a range of num_steps between min and max values. However if the required step size
    is smaller than the given min_step_size, reduce the number of steps to align with the min
    step size. The range returned is always bounded by the mix/max values given

    :param min_value (float): the range start value
    :param max_value (float): the range end value
    :param num_steps (float): the number of steps to break the range into
    :param min_step_size (float): the minimum step size
    :return: a numpy array of the required steps

    """
    assert max_value > min_value, "Max must be greater then min"
    assert num_steps > 1, "Number of steps must be greater than one"

    step_size = (max_value - min_value) / (num_steps - 1)
    if min_step_size is not None:
        if step_size < min_step_size:
            step_size = min_step_size
            num_steps = 1 + int((max_value - min_value) / step_size)

    rng = []
    for i in range(num_steps):
        rng.append(min_value + (i * step_size))

    return rng

In [ ]:
def parameter_scenarios(
    vol_range: Tuple[float, float] = (
        MarketConstants.volMin,
        MarketConstants.volMax),
    mat_range: Tuple[float, float] = (
        MarketConstants.timeMin,
        MarketConstants.timeMax),
    spot_range: Tuple[float, float] = (
        MarketConstants.spotMin,
        MarketConstants.spotMax,),
    strike_range: Tuple[float, float] = (
        MarketConstants.strikeMin,
        MarketConstants.strikeMax),
    rate_range: Tuple[float, float] = (
        MarketConstants.rateMin,
        MarketConstants.rateMax),
    num_steps=50,
    simple=True
) -> Tuple[List[float], List[float], List[float], List[float], List[float]]:
    """
    Create scenario vectors for volatility, maturity, spot, strike and risk-free rate

    Note: In practice these parameters have natural upper and lower limits. This helps as we reduce the possible span
          of feature space that we need to train in.

    :param vol_range: The range to vary volatility over
    :param mat_range: The range to vary maturity over
    :param spot_range: The range to vary spot over
    :param strike_range: The range to vary strike over
    :param rate_range: The range to vary risk-free rate over
    :param num_steps: number of scenario steps
    :return: volatility, maturity, spot, strike and risk-free rate scenarios as list of float
    """
    vols = stepped_range(
        vol_range[0], vol_range[1], num_steps, None if simple else MarketConstants.volMinStep
    )
    mats = stepped_range(
        mat_range[0], mat_range[1], num_steps, None if simple else MarketConstants.timeMinStep
    )
    spots = stepped_range(
        spot_range[0], spot_range[1], num_steps, None if simple else MarketConstants.spotMinStep
    )
    strikes = stepped_range(
        strike_range[0], strike_range[1], num_steps, None if simple else MarketConstants.strikeMinStep
    )
    rates = stepped_range(
        rate_range[0], rate_range[1], num_steps, None if simple else MarketConstants.rateMinStep
    )

    return vols, mats, spots, strikes, rates

In [ ]:
def combinations(
    list1: Union[List[float], float],
    list2: Union[List[float], float]
) -> List:
    """
    Return all combinations of the given lists

    :param list1: The first list
    :param list2: The second list
    :return: List of all combinations of lists 1 and List 2
    """
    if not isinstance(list1, List):
        list1 = [list1]
    if not isinstance(list2, List):
        list2 = [list2]
    res = []
    for x in list1:
        for y in list2:
            if not isinstance(x, List):
                x = [x]
            if not isinstance(y, List):
                y = [y]
            res.append([*x, *y])
    return res

In [ ]:
def generate_model_scenarios(
    spot: Union[List[float], float],
    strike: Union[List[float], float],
    mat: Union[List[float], float],
    rate: Union[List[float], float],
    vol: Union[List[float], float],
) -> List[Tuple[float, float, float, float, float]]:
    """
    Return a set of model scenario inputs based on given parameter scenarios

    :param vol: List of volatilities or single volatility
    :param vol: List of maturities or single maturity
    :param vol: List of spots or single spot
    :param vol: List of strike or single strike
    :param vol: List of rates or single rate
    :return: List of all combinations [[s,k,t,r,v]]
    """
    return combinations(
        combinations(combinations(combinations(spot, strike), mat), rate), vol
    )


In [ ]:
def two_d_scenario(
    spot: Union[List[float], float],
    strike: Union[List[float], float],
    mat: Union[List[float], float],
    rate: Union[List[float], float],
    vol: Union[List[float], float],
    price_func: Callable,
    scaler=None,
) -> Tuple[List[float], str, List[float], str, np.ndarray]:
    """
    Generate a scenario where two of the given parameters are scenarios

    :param spot: spot scenario or single spot
    :param strike: strike scenario or single strike
    :param maturity: maturity scenario or single maturity
    :param rate: rate scenario or single rate
    :param vol: volatility scenario or single volatility
    :param price_func: The pricing function, either black_scholes or model.predict
    :param scaler: The scaler used to normalize the model inputs
    :return: scenario List 1, scenario parameter 1 name, scenario List 2, scenario parameter 2 name, scenario prices

    """
    params = [spot, strike, mat, rate, vol]
    arg_names = [*inspect.signature(price_func).parameters.keys()]
    scenario_params = [
        [x[0][0], x[0][1], x[1]]
        for x in zip(enumerate(params), arg_names)
        if isinstance(x[0][1], List)
    ]
    assert (
        len(scenario_params) == 2
    ), "Only two parameters can be passed as scenario lists"
    prices = np.zeros((len(scenario_params[0][1]), len(scenario_params[1][1])))
    for i, x in enumerate(scenario_params[0][1]):
        if price_func.__name__ != "predict":  # cell by cell
            for j, y in enumerate(scenario_params[1][1]):
                params[scenario_params[0][0]] = x
                params[scenario_params[1][0]] = y
                prices[i, j] = (price_func)(*params)[BSConstants.price]
        else:  # row by row as it is quicker when calling model.predict
            params[scenario_params[0][0]] = x
            params[scenario_params[1][0]] = None
            param_set = np.tile(np.asarray(params), (len(scenario_params[1][1]), 1))
            param_set[:, scenario_params[1][0]] = scenario_params[1][1]
            if scaler is not None:
                param_set = scaler.transform(param_set)
            prices[i, :] = (
                (price_func)(param_set.astype(np.float64), verbose=0)
            ).reshape(1, len(scenario_params[1][1]))
        print(
            "{:0.0f} % Complete".format(
                100 * ((i * len(scenario_params[1][1])) / prices.size)
            )
        )
    print("Done")
    return (
        scenario_params[0][1],
        scenario_params[0][2],
        scenario_params[1][1],
        scenario_params[1][2],
        prices,
    )


### Generate parameter scenarios

We don't have a database of previous quotes to use, so we will generate a range for each model parameter type. These will be used to generate training data by calling the model for each combination of the parameters. In this *toy* example we have the luxury of creating all the training data we need and ensuring that it is evenly distributed thought the feature space.

In a real case we would try to collect this data as a side effect of where the real model was being used and only augment the training set where we observed gaps or thin spots in the training data. So with the real case, we would have to investigate the coverage of existing data in feature space and plug gaps as needed. We would also need to check for systematic shifts in the valuation parameters e.g. higher or lower prices, significant changes oin volatility, rates etc. In these case we would have to ensure the model was re-trained.

Justing periodic re-training with data generated as part of RFQ process would not always be sufficient, especially if there were sudden shifts in the market.

In [ ]:
#
# Generate a spread of parameter values - the will form the X vector of training data.
#
# Note that even for this toy example, we already have 3.2 million training examples, even for a relatively course grid over feature space.
#
vols, mats, spots, strikes, rates = parameter_scenarios(num_steps=20)
X = generate_model_scenarios(spot=spots, strike=strikes, mat=mats, rate=rates, vol=vols)
print(f"Generated {len(X):,} training scenarios")

In [ ]:
#
# Value the option for each training X case, this will generate the Y value for training.
#
Y = [black_scholes_model(*params)[BSConstants.price] for params in X]

In [ ]:
#
# Shuffle training data, to prevent model getting stuck in local optima in a region of state space.
#
Xs, Ys = shuffle(X, Y, random_state=42)

In [ ]:
#
# Merge results in a Panda frame so it is easier to manipulate
#
dfy = pd.DataFrame(Ys, columns=["price"])
dfx = pd.DataFrame(Xs, columns=["vols", "mats", "spots", "strikes", "rates"])
dfRaw = dfx.join(dfy)

In [ ]:
#
# Save the raw data, so we can use later.
#
dfRaw.to_csv("XYRawSimple.csv", encoding="ascii", index=False)

### A regression Neural network.

This is where art meets science in that there are no hard and fast rules for the size, shape and architecture of a neural network that will be able to converge on a general form of the function in a given set of data (if one exists). There are guidelines and many types of layer that fit certain patterns for image processing etc, so there is always informed experimentation at this stage.

This is a very simple model as the data set is relatively small, however even a simple data set such as this took a number of experiments to get the right balance.

The model below has relatively few trainable parameters, but is deep (layers) by comparison as we need to capture the complexities function shape in all five dimensions of the feature (parameter) space. 

I did experiment with layer size, drop-outs etc, but in the end reducing the layer sizes and adding more layers gave the best result. With larger layer sizes the model
had a tendency to over fit even when dropputs were added. For optimisation a standard Adam Optimiser and mean squared error as loss function, with a slight modified learning rate also gave a reasonable result. 

In [ ]:
def create_neural_net():
    """
    Create a Neural network with an architecture tuned to regression.

    We chose a simple dense neural network here that is powerful enough to capture
    a generalized form, but not too power full as to 'learn'the data and over fit.

    """

    # Create a NN of 5 Dense layers, with 5 inputs (number of BS parameters) abd a single
    # output that predicts the price for the given parameters
    model = Sequential()

    model.add(
        Dense(32, input_shape=(5,), kernel_initializer="normal", activation="tanh")
    )
    model.add(Dense(16, kernel_initializer="normal", activation="tanh"))
    model.add(Dense(8, kernel_initializer="normal", activation="tanh"))
    model.add(Dense(4, kernel_initializer="normal", activation="tanh"))
    model.add(Dense(2, kernel_initializer="normal", activation="tanh"))
    model.add(Dense(1, kernel_initializer="normal", activation="linear"))

    # Compile the model with the Adam optimizer, with a tuned step size.
    opt = tf.keras.optimizers.Adam(learning_rate=0.0005)
    model.compile(loss="mean_squared_error", optimizer=opt)

    return model

In [ ]:
def create_deeper_neural_net():
    """
    Create a Neural network with an architecture tuned to regression and able to capture (generalize)
    more complex features of closed-form model + barriers.

    We chose a simple dense neural network here that is powerful enough to capture
    a generalized form, but not too power full as to 'learn'the data and over fit.

    """

    # Create a NN of 5 Dense layers, with 5 inputs (number of BS parameters) abd a single
    # output that predicts the price for the given parameters
    model = Sequential()

    model.add(
        Dense(32, input_shape=(5,), kernel_initializer="normal", activation="tanh")
    )
    model.add(Dense(16, kernel_initializer="normal", activation="tanh"))
    model.add(Dense(8, kernel_initializer="normal", activation="tanh"))
    model.add(Dense(8, kernel_initializer="normal", activation="tanh"))
    model.add(Dense(4, kernel_initializer="normal", activation="tanh"))
    model.add(Dense(4, kernel_initializer="normal", activation="tanh"))
    model.add(Dense(2, kernel_initializer="normal", activation="tanh"))
    model.add(Dense(2, kernel_initializer="normal", activation="tanh"))
    model.add(Dense(1, kernel_initializer="normal", activation="linear"))

    # Compile the model with the Adam optimizer, with a tuned step size.
    opt = tf.keras.optimizers.Adam(learning_rate=0.0005)
    model.compile(loss="mean_squared_error", optimizer=opt)

    return model

### Fit the data

Fitting can be switched between a standard dense network and a random forest. Both will learn to predict the price from the inputs with different internal approaches. We will investigate the differences when we look at how the predictions compare to the ideal as defined by the model we are fitting.


In [ ]:
class ModelType(Enum):
    NEURAL_NET = 1
    DEEP_NEURAL_NET = 2
    RANDOM_FOREST_REGRESSOR = 3


def fit(X: np.ndarray, Y: np.ndarray, model_type: ModelType, num_epoch: int = 50):
    """
    Prepare the model inputs and fit the data with the given model type.

    :param X: Pricing parameter scenarios
    :param Y: Prices corresponding to given inputs
    :param model_type: NEURAL_NET or RANDOM_FOREST_REGRESSOR
    :return : The tf model, training history & test and training data used X_train, Y_train, X_test, Y_test

    """

    # Split data into train & test

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)
    X_train = np.asarray(X_train)
    X_test = np.asarray(X_test)
    Y_train = np.asarray(Y_train)
    Y_test = np.asarray(Y_test)

    # Create a check point to save the model version with the best validation loss

    saveBest = ModelCheckpoint(
        filepath="best", monitor="val_loss", verbose=1, save_best_only=True, mode="min"
    )

    # Fit to the requested type

    if model_type == ModelType.NEURAL_NET:
        print("Create and fit Neural Network")
        model = create_neural_net()
        history = model.fit(
            X_train,
            Y_train,
            batch_size=256,
            epochs=num_epoch,
            verbose=1,
            validation_data=(X_test, Y_test),
            callbacks=[saveBest],
        )
        model.load_weights(
            "best"
        )  # load back the model corresponding to lowest validation loss
    elif model_type == ModelType.DEEP_NEURAL_NET:
        print("Create and fit Deep Neural Network")
        model = create_deeper_neural_net()
        history = model.fit(
            X_train,
            Y_train,
            batch_size=256,
            epochs=num_epoch,
            verbose=1,
            validation_data=(X_test, Y_test),
            callbacks=[saveBest],
        )
        model.load_weights(
            "best"
        )  # load back the model corresponding to lowest validation loss
    elif model_type == ModelType.RANDOM_FOREST_REGRESSOR:
        print("Create and fit Random Forest Regressor")
        model = RandomForestRegressor(n_estimators=250, verbose=2)
        model.fit(X_train, Y_train)
        history = None
    else:
        assert (
            False
        ), "Invalid model type specified, see model types defined by class ModelType"

    mse = mean_squared_error(model.predict(X_test), Y_test)
    print(f"Final Mean Square Error [{mse:5f}]")
    return model, history, X_train, Y_train, X_test, Y_test

### Train the model by fitting to the given training data

**Note** the neural network will mak predictions even **before** it is trained and at any stage during the training process. From the point of view of the neural network it does not return a confidence level in its prediction, our confidence in its predictions is based on us having trained the model on a representative set of data an achieved an task appropriate balance between fit and generalisation. This we can assess from the lowest loss and validation loss achieved as well as specific testing.

An extension of this idea is that the neural network will make predictions for scenarios it has not been trained on, this happens when our feature space changes and we do not retain the model. Again the neural network will not generate an error or complain when it is asked to make a prediction outside of its experience. It falls to us to catch the need to retrain or prevent predictions from outside of its experience being requested by our application.

So, before we train the model we will run some predictions so we can see this effect.

In [ ]:
# Un-trained model, for demo.
#
model = create_neural_net()
model.load_weights("best")

In [ ]:
# Partially train from shuffled data Xs, Ys - with limited epochs
#
model, history, X_train, Y_train, X_test, Y_test = fit(
    Xs, Ys, ModelType.NEURAL_NET, num_epoch=3
)

In [ ]:
# Train from shuffled data Xs, Ys
#
model, history, X_train, Y_train, X_test, Y_test = fit(
    Xs, Ys, ModelType.DEEP_NEURAL_NET, num_epoch=100
)

### Examine the training losses and verify if model has learned a generalisation of the option function.

We expect to see the Loss (red line) drop off exponentially as the model learns (fits) the data and the error between actual and predicted reduces.

The validation loss (blue line) should also drop off as the model get better at predicting for test data (hold out) that it has not seen during training. If the validation loss increases, this is an indication that the model is too powerful and has just learned the training data (over fitted) rather than fitting a generalized form of the function in the training data.

The final validation loss is an indication of how accurate the model is. The accuracy is critical in determining if the model can be used in a commercial setting, however we must do more exhaustive testing to make sure the accuracy is preserved across the feature space.

Note: the random forest regressor, does not have a training history as it works in a fundamentally different way.

In [ ]:
def plot_training_history(
    loss: List[float], validation_loss: List[float], skip: int = 0
) -> None:
    """
    Plot a two axis line graph of training and validation losses

    :param loss            : Training losses per epoch
    :param validation_ loss: Validation losses per epoch
    :param skip            : don't plot the first skip points, this is because early loss values
                             are typically and order of magnitude or two bigger than than the final
                             ones. As such when plotted together it is not possible to see how the
                             loss and validation loss behaved.
    """
    _, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    ax1.set_xlabel("Training Epoch")
    ax1.set_ylabel("Loss", color="r")
    ax2.set_ylabel("Validation Loss", color="b")
    ax1.plot(loss[skip:], color="r")
    ax2.plot(validation_loss[skip:], color="b")
    plt.show()
    return


In [ ]:
#
# Random forest does not generate a training history
#
if history is not None:
    plot_training_history(history.history["loss"], history.history["val_loss"], skip=8)

### Verify the operation of the model

We must verify the model is working across feature space. To do this we create a new set of scenarios and as the model to predict, we then compare to values generated by the original mathematical version. 

In [ ]:
# Create new set of test scenarios
#
vols, mats, spots, strikes, rates = parameter_scenarios(num_steps=20)

**Note**
There are three scenarios below, only run **one** of them at a time as they all return results to the same outputs. 
These outputs are used to plot the graphs that inspect the accuracy of the results. You can come back and run a different scenario and re-plot.

In [ ]:
# Scenario 1 : Rate vs Vol Scenario
#
vols, mats, spots, strikes, rates = parameter_scenarios(num_steps=100)
s1, s1n, s2, s2n, actual = two_d_scenario(
    spot=99, strike=100, mat=0.75, rate=rates, vol=vols, price_func=black_scholes_model
)
s1, s1n, s2, s2n, predicted = two_d_scenario(
    spot=99, strike=100, mat=0.75, rate=rates, vol=vols, price_func=model.predict
)

In [ ]:
# Scenario 2 : Maturity vs Vol Scenario
#
vols, mats, spots, strikes, rates = parameter_scenarios(num_steps=100)
s1, s1n, s2, s2n, actual = two_d_scenario(
    spot=99, strike=100, mat=mats, rate=0.05, vol=vols, price_func=black_scholes_model
)
s1, s1n, s2, s2n, predicted = two_d_scenario(
    spot=99, strike=100, mat=mats, rate=0.05, vol=vols, price_func=model.predict
)

In [ ]:
# Scenario 3 : Spot vs Strike
#
vols, mats, spots, strikes, rates = parameter_scenarios(num_steps=100)
s1, s1n, s2, s2n, actual = two_d_scenario(
    spot=spots,
    strike=strikes,
    mat=1.0,
    rate=0.05,
    vol=0.2,
    price_func=black_scholes_model,
)
s1, s1n, s2, s2n, predicted = two_d_scenario(
    spot=spots, strike=strikes, mat=1.0, rate=0.05, vol=0.2, price_func=model.predict
)

In [ ]:
def plot_price_surface(
    xscen: np.ndarray,
    yscen: np.ndarray,
    actual: np.ndarray,
    predicted: np.ndarray,
    title: str,
    xscen_lab: str,
    yscen_lab: str,
    azimuth: float = -45,
    elevation: float = 30,
) -> None:
    """
    Plot dual surface of actual vs predicted, with a contour plot of price difference as %

    :param xscen     : The scenario values for X Axis
    :param yscen     : The scenario values for Y Axis
    :param actual    : 2D Array of actual values for X and Y Scenarios
    :param predicted : 2D Array of neural net predicted values for X and Y Scenarios
    :param title     : Graph title
    :param xscen_lab : X Axis label
    :param yscen_lab : Y Axis label
    """
    minz = min(np.min(actual), np.min(predicted))
    maxz = max(np.max(actual), np.max(predicted))

    X, Y = np.meshgrid(xscen, yscen)
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection="3d")
    ax.set_zlim3d(minz, maxz)
    c1 = ax.contourf(
        X,
        Y,
        ((actual - predicted) / np.maximum(1e-9, actual)) * 100.0,
        levels=range(-100, 100, 1),
        cmap=cm.RdGy,
        offset=np.min(minz),
    )
    _ = ax.plot_surface(
        X, Y, actual, cmap=cm.coolwarm, linewidth=0, edgecolor="none", alpha=0.7
    )
    _ = ax.plot_surface(
        X, Y, predicted, edgecolors="k", linewidth=0.1, color="gray", alpha=0.3
    )
    cax = fig.add_axes(
        [
            ax.get_position().x1 + 0.05,
            ax.get_position().y0 + ((ax.get_position().height) * 0.15),
            0.02,
            (ax.get_position().height) * 0.7,
        ]
    )
    cb = fig.colorbar(c1, cax=cax)
    cb.ax.set_ylabel("% difference", rotation=270)
    ax.set_xlabel(xscen_lab)
    ax.set_ylabel(yscen_lab)
    ax.set_zlabel("Option Price")
    ax.set_title(title)
    ax.view_init(azim=azimuth, elev=elevation)
    plt.show()

## Actual vs Predicted

### Surface

We plot the actuals as a surface (blue-red) and then overlay the predicted as a translucent gray surface, so we can see where the two diverge. We also plot a contour of the % difference at every point.

In [ ]:
# Plot results from one of the last scenarios above that was run.
#
plot_price_surface(
    s1,
    s2,
    actual=actual,
    predicted=predicted,
    title="Predicted vs Actual Comparison",
    xscen_lab=s1n,
    yscen_lab=s2n,
)

### Scatter 
If we scatter plot actual vs predicted then we would expect to see a tight line, where the actual and predicted are the same for given inputs. So the more diffused the line the lower the fit between actual and predicted.

In [ ]:
def plot_scatter(
    actual: List[float], predicted: List[float], title: str, x_lab: str, y_lab: str
) -> None:
    """
    Plot a scatter of actual vs predicted as a way to visualize differences.

    :param actual    : 1D Array of actual priced
    :param predicted : 1D Array of predicted
    :param title     : Graph title
    :param x_lab     : X Axis label
    :param y_lab     : Y Axis label
    """
    plt.figure(figsize=(10, 10))
    plt.title(title)
    plt.xlabel(x_lab)
    plt.ylabel(y_lab)
    plt.scatter(actual.flatten(), predicted.flatten(), s=0.25)
    plt.show()
    return

In [ ]:
# Plot actual vs predicted (if you plot actual/actual or predicted/predicted - you will see s straight line)
#
plot_scatter(
    actual.flatten(),
    predicted.flatten(),
    x_lab="Actual",
    y_lab="Predicted",
    title="Accuracy Comparison",
)

### Option Greeks, specifically Delta

When trading options we need to know how the option behaves as the parameters change, Spot, Volatility etc .. these sensitivities to parameter changes are referred to as options greeks as they are named delta, gamma, vega, rho etc.

A critical one to know is delta, which is the rate of change of the option price as the spot price changes. For the closed form model we are using here, there is a closed form function for delta. However in the more complex options delta is calculated numerically. This is done by calculating an option price for a given spot level, and a the option price foe very small increase in spot. (very small as in 1/1'000'000). From this we can calculate the rate of change of option price with respect to change in spot.

We can use our trained network to generate the numerical delta and compare it to the closed form delta, this will be another indication of the viability of the trained network.

In [ ]:
numerical_deltas = []
closed_form_deltas = []

small_change: Final[float] = 1e-6

# Spot range must be within the trained range
spots = range(1, 100)
x_p1 = np.zeros((len(spots), 5))
x_p2 = np.zeros((len(spots), 5))

for i, spot in enumerate(spots, 0):
    # Calc the closed form delta
    _, delta = black_scholes_model(
        S=spot, K=50, T=1.0, r=0.05, v=0.25, with_delta=True
    )
    closed_form_deltas.append(delta)
    
    # Build up the scenarios to make batch calls to model predict outside of this loop.
    x_p1[i] = [spot, 50, 1.0, 0.05, 0.25]
    x_p2[i] = [spot + small_change, 50, 1.0, 0.05, 0.25]

# Predict as batches
p1 = model.predict(x_p1, verbose=0)
p2 = model.predict(x_p2, verbose=0)

# Numerical delta is diff in price over spot increment
numerical_deltas = ((p2-p1)/small_change).tolist()

# Plot the results vs the closed form to make it easy to visualize
plt.clf()
plt.title(f"Numerical NN Delta vs Closed form Delta")
plt.plot(spots, closed_form_deltas, color="blue", label="Closed Form")
plt.plot(spots, numerical_deltas, color="red", label="NN Numeric")
plt.legend()
plt.show()

## Conclusion

We can see that it is possible to train a neural network for a financial contracts with a well behaved pay-off. We now add in an arbitrary feature that makes the pay-off harder to predict and type again.

The acid test, is that if we quoted options from the Neural Network, would we be over/under pricing, in which case we would be exposing an arbitrage opportunity to the market. In this case the market would be able to make mostly risk free profit from trading with us by monetising the miss-pricing. This would systematically costs us money to trade and ultimately we would be driven out of business.

### Best Case ...

We need to take a moment to consider the <sup>1</sup> joke below

> There’s this farmer, and he has these chickens, but they won’t lay any eggs. So, he calls a physicist to help. The physicist then does some calculations, and he says, um, I have a solution, but it only works with spherical chickens in a vacuum

Our *spherical chicken in a vacuum* is the Nobel winning [Black Scholes Model](https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model), while ground breaking, out of the box it represents a best case, where our option is a spherical chicken in a vacuum, with a well behaved pay off. In practice there are many embellishments required to accommodate dividends, early exercise, barriers, cross currency (quanto) etc. These embellishments make the pay off less well behaved in mathematical terms. This is relevant to us, as approximating functions with complex, sharp or instantaneous transitions with a large number of parameters is harder then approximating functions with a small number of parameters and smooth transitions. The real world use case where an options have a barrier conditions, where within a given range of pricing inputs the option behavior changes, sometimes instantaneously to zero value.

We can gain an intuition for this complexity by looking at Fourier transforms, which is another way of approximating functions. There is a way of modelling a sharp transition (step function) using Fourier transforms. However to effectively model this instantaneous transition we must allow ourselves to use functions of infinitely high frequency. Clearly this is not practical, so limit ourselves to finite frequencies that yield a workable level of accuracy. There is a similar trade off with neural networks, we cannot have infinitely complex networks, we must try to find the least complexity that can capture and generalise the payoff with a tradable level of accuracy and precision.

In the cells below, we show how fourier transforms work and compare five frequency terms versus one hundred frequency terms. Where more terms give us a very close approximation, but still an approximation.

The analogy with neural networks is that as the real world complexity rises, we require a significant amount of data and deep networks to capture the payoff.

So, let's have a look at the Fourier transform as a means to gain some intuition into this challenge.

<sup>1</sup> *Credit [Big Bang Theory](https://isntsciencewonderful.wordpress.com/2015/09/06/hofstadters-spherical-chickens/)*

In [ ]:
def fourier_approx_of_step_function(num_fourier_terms: int = 5):
    """
    Calculate the first n terms of the fourier transform that approximates a cyclic step function.

    |---|   |---|
    |   |   |   |
    |   |---|   |---

    https://en.wikipedia.org/wiki/Fourier_series

    Keyword Arguments:
        num_fourier_terms -- number of fourier terms to include (default: {5})
    """
    num_increments = 100
    vals = np.asarray([(x / 100.0) * 4 * np.pi for x in range(num_increments)])
    res = np.asarray(
        [
            [(4 * np.sin(n * (v))) / (np.pi * n) for v in vals]
            for n in range(1, (num_fourier_terms * 2), 2)
        ]
    )
    return vals, res, res.sum(axis=0)

In [ ]:
def plot_step_fnx(num_fourier_terms: int = 5):
    """
    Calculate and plot all of the sub terms of the Fourier transform for a step function.
    Plot the sub terms in grey & the final approximation (sum) in Red

    Keyword Arguments:
        num_fourier_terms -- number of fourier terms to include (default: {5})
    """
    vals, res, apx = fourier_approx_of_step_function(
        num_fourier_terms=num_fourier_terms
    )
    plt.clf()
    plt.title(f"{num_fourier_terms} terms")
    for i, r in enumerate(res):
        plt.plot(vals, r, color="gray", linewidth=max(0.0125, np.power(0.7, i)))
    plt.plot(vals, apx, color="red", linewidth=1)
    plt.show()
    return

In [ ]:
# We see the higher the number of terms the more closely we can approximate the step function.
# The implications are that the neural network has to work *harder* to approximate functions with
# sharp transitions. Where there sharp transitions are very common in the day to day of financial options
#
# In grey we can see all of the sub-terms, in red we can see the shape of the function we get if we add
# the sub terms together
#
plot_step_fnx(5)
plot_step_fnx(100)

So, now we have an intuition for the challenge in capturing complexity, we will add real world features to our option and see what we need to do the neural network model and supply of training data to try and capture this more complex form.

We start by adding barrier conditions, where the payoff changes for a defined range of inputs. We compare two types of barrier, one with smooth transitions and one with instantaneous ones, the latter, as above we expect to be harder to capture. 

### Smooth Barrier

We now model a barrier that has a rapid but smooth transition for the option price.

In [ ]:
# These settings are arbitrary, and just chosen to place the barrier in the center
# of the range of spot prices, so it is easy to visualise when graphed.
#
# In practice, the barriers are normally selected by the buyer to fit a strategy they
# have in mind to protect against loss of speculate against expected market moves.
#
vols, mats, spots, strikes, rates = parameter_scenarios(num_steps=20)

price_max = spots[-1]
price_min = spots[0]
barrier_min = (
    price_max - price_min
) * 0.45  # Barrier kicks in at 45% of the max option price
barrier_max = (
    price_max - price_min
) * 0.8  # Barrier end at 80% of the max option price

In [ ]:
def smooth_barrier(
    p: float,
    S: float,
    v: float,
    T: float,
    barrier_spot_min: float,
    barrier_spot_max: float,
) -> float:
    """
    Takes the current option price and reduces it smoothly, but sharply if the spot price in the the min/bax barrier range.

    :param p: Unmodified option price
    :param s: spot used to generate the given unmodified option price
    :param v: vol used to generate the given unmodified option price
    :param T: Maturity used to generate the given unmodified option price
    :param barrier_spot_min: The spot price above which the barrier operator is active
    :param barrier_spot_max: The spot price below which the barrier operator is active
    :return: Modified option price.
    """
    scale_down = float(1)
    if S >= barrier_spot_min and S <= barrier_spot_max:
        scale_down = 1 - np.sin(
            ((S - barrier_spot_min) / (barrier_spot_max - barrier_spot_min)) * np.pi
        )

    return p * scale_down

In [ ]:
# Pre-fill the additional params to barrier, so function conforms with our defined barrier prototype
#
smb = partial(
    smooth_barrier, barrier_spot_min=barrier_min, barrier_spot_max=barrier_max
)

In [ ]:
# Run quick simulation to show shape of barrier
#
opt_prc = []
spot_prc = []
for s in range(1, 100):
    opt_prc.append(
        black_scholes_model(S=s, K=10, T=1.0, r=0.05, v=0.3, o=0, barrier=smb)
    )
    spot_prc.append(s)

In [ ]:
def plot_line(
    x: List[float], y: List[float], title: str = "", x_lab: str = "", y_lab: str = ""
) -> None:
    """
    Simple line plot of a single variable.

    :param x         : 1D Array of x values to plot.
    :param y         : 1D Array of y values to plot.
    :param title     : Graph title
    :param x_lab     : X Axis label
    :param y_lab     : Y Axis label
    """
    plt.figure(figsize=(5, 5))
    plt.title(title)
    plt.xlabel(x_lab)
    plt.ylabel(y_lab)
    plt.plot(x, y)
    plt.show()
    return

In [ ]:
# Show the pay-off, with the addition of smooth barrier.
# We can see the payoff drop when the price in within the barrier range.
#
plot_line(
    x=spot_prc,
    y=opt_prc,
    title="Smooth Barrier - pay-off",
    x_lab="Spot Price",
    y_lab="Option Price",
)

In [ ]:
#
# Generate a fresh set of training data for barrier type 1 = smooth barrier.
#

X_b1 = generate_model_scenarios(
    spot=spots, strike=strikes, mat=mats, rate=rates, vol=vols
)
print(f"Generated {len(X_b1):,} training scenarios")
Y_b1 = [black_scholes_model(*params, barrier=smb) for params in X_b1]
Xs_b1, Ys_b1 = shuffle(X_b1, Y_b1, random_state=42)

In [ ]:
#
# Now fit the new training data with the same model architecture and see if it can (cope) generalise the more complex pay off.
#
model_b1, history_b1, X_train_b1, Y_train_b1, X_test_b1, Y_test_b1 = fit(
    Xs_b1, Ys_b1, ModelType.NEURAL_NET
)
if history_b1 is not None:
    plot_training_history(
        history_b1.history["loss"], history_b1.history["val_loss"], skip=8
    )

In [ ]:
# Calculate : Spot vs Vol for Smooth barrier
#
def bsmb(S: float, K: float, T: float, r: float, v: float, o: float = float(0)):
    return black_scholes_model(S, K, T, r, v, o, barrier=smb)


vols, mats, spots, strikes, rates = parameter_scenarios(num_steps=100)
s1, s1n, s2, s2n, actual = two_d_scenario(
    spot=spots, strike=20.0, mat=1.0, rate=0.05, vol=vols, price_func=bsmb
)
s1, s1n, s2, s2n, predicted = two_d_scenario(
    spot=spots,
    strike=20.0,
    mat=1.0,
    rate=0.05,
    vol=vols,
    price_func=model_b1.predict,
    scaler=None,
)

In [ ]:
plot_price_surface(
    vols,  # Can swap these next two to see different scenarios
    mats,
    actual=actual,
    predicted=predicted,
    title="Predicted vs Actual Comparison",
    xscen_lab=s1n,
    yscen_lab=s2n,
)
plot_scatter(
    actual.flatten(),
    predicted.flatten(),
    x_lab="Actual",
    y_lab="Predicted",
    title="Accuracy Comparison",
)

In [ ]:
def plot_dual_line(
    x: List[float],
    y1: List[float],
    y2: List[float],
    title: str = "",
    x_lab: str = "",
    y1_lab: str = "",
    y2_lab: str = "",
) -> None:
    """
    Simple dual line plot.

    :param x         : Common 1D Array of x values to plot for y1 and y2
    :param y1        : First 1D Array of y values to plot.
    :param y2        : Second 1D Array of y values to plot.
    :param title     : Graph title
    :param x_lab     : X Axis label
    :param y1_lab    : y1 Axis label
    :param y2_lab    : y2 Axis label
    """
    fig, ax1 = plt.subplots(figsize=(8, 8))
    ax2 = ax1.twinx()
    ax1.set_title(title)
    ax1.set_xlabel(x_lab)
    ax1.plot(x, y1, color="green")
    ax2.plot(x, y2, color="red")
    ax1.set_ylabel(y1_lab)
    ax2.set_ylabel(y2_lab)
    ax1.legend([y1_lab], loc=0)
    ax2.legend([y2_lab], loc=9)
    plt.tight_layout()
    plt.show()
    return

In [ ]:
# Run a simulation to show shape of predicted price with barrier
#
opt_prc_pred = []
spot_prc = []
for s in range(1, 100):
    xs = np.asarray([s, 10, 1.0, 0.05, 0.3]).reshape((1, 5))
    opt_prc_pred.append(model_b1.predict(xs, verbose=0)[0])
    spot_prc.append(s)

plot_dual_line(
    x=spot_prc,
    y1=opt_prc,
    y2=opt_prc_pred,
    title="Smooth Barrier Actual vs Predicted",
    x_lab="Spot Price",
    y1_lab="Actual Option Price",
    y2_lab="Predicted Option Price",
)

### Step Barrier

We now try training with a barrier that has a sharp transition to zero, instantaneous transitions are difficult to model but are common in financial options.

In [ ]:
def step_barrier(
    p: float,
    S: float,
    v: float,
    T: float,
    barrier_spot_min: float,
    barrier_spot_max: float,
) -> float:
    """
    Takes the current option price and reduces it to zero between spot min and max

    :param p: Unmodified option price
    :param s: spot used to generate the given unmodified option price
    :param v: vol used to generate the given unmodified option price
    :param T: Maturity used to generate the given unmodified option price
    :param barrier_spot_min: The spot price above which the barrier operator is active
    :param barrier_spot_max: The spot price below which the barrier operator is active
    :return: Modified option price.
    """
    scale_down = float(1)
    if S >= barrier_spot_min and S <= barrier_spot_max:
        scale_down = float(0)

    return p * scale_down

In [ ]:
# Pre-fill the additional params to barrier, so function conforms with expected prototype
#
sms = partial(step_barrier, barrier_spot_min=barrier_min, barrier_spot_max=barrier_max)

In [ ]:
# Run quick simulation to show shape of barrier
#
opt_prc = []
spot_prc = []
for s in range(1, 100):
    opt_prc.append(
        black_scholes_model(S=s, K=10, T=1.0, r=0.05, v=0.3, o=0, barrier=sms)
    )
    spot_prc.append(s)

plot_line(
    x=spot_prc,
    y=opt_prc,
    title="Step Barrier",
    x_lab="Spot Price",
    y_lab="Option Price",
)

In [ ]:
#
# Generate a fresh set of training data for barrier type 2 = step barrier.
#
vols, mats, spots, strikes, rates = parameter_scenarios(num_steps=25)
X_b2 = generate_model_scenarios(
    spot=spots, strike=strikes, mat=mats, rate=rates, vol=vols
)
print(f"Generated {len(X_b2)} training scenarios")
Y_b2 = [black_scholes_model(*params, barrier=sms) for params in X_b2]
Xs_b2, Ys_b2 = shuffle(X_b2, Y_b2, random_state=42)

In [ ]:
#
# Now fit the new training data with the same model architecture and see if it can (cope) generalise the more complex pay off.
#

# model_b2, history_b2, X_train_b2, Y_train_b2, X_test_b2, Y_test_b2 = fit(
#    Xs_b2, Ys_b2, ModelType.NEURAL_NET)

model_b2, history_b2, X_train_b2, Y_train_b2, X_test_b2, Y_test_b2 = fit(
    Xs_b2, Ys_b2, ModelType.DEEP_NEURAL_NET
)
if history_b2 is not None:
    plot_training_history(
        history_b2.history["loss"], history_b2.history["val_loss"], skip=8
    )

In [ ]:
# Calculate : Spot vs Vol for Step Barrier
#
def bsms(S: float, K: float, T: float, r: float, v: float, o: float = float(0)):
    return black_scholes_model(S, K, T, r, v, o, barrier=sms)


vols, mats, spots, strikes, rates = parameter_scenarios(num_steps=100)

s1, s1n, s2, s2n, actual = two_d_scenario(
    spot=spots, strike=20.0, mat=1.0, rate=0.05, vol=vols, price_func=bsms
)
s1, s1n, s2, s2n, predicted = two_d_scenario(
    spot=spots,
    strike=20.0,
    mat=1.0,
    rate=0.05,
    vol=vols,
    price_func=model_b2.predict,
    scaler=None,
)

In [ ]:
plot_price_surface(
    vols,  # Can swap these next two to see different scenarios
    mats,
    actual=actual,
    predicted=predicted,
    title="Predicted vs Actual Comparison",
    xscen_lab=s1n,
    yscen_lab=s2n,
)

plot_scatter(
    actual.flatten(),
    predicted.flatten(),
    x_lab="Actual",
    y_lab="Predicted",
    title="Accuracy Comparison",
)

In [ ]:
# Run a simulation to show shape of predicted price with barrier
#
opt_prc_pred = []
spot_prc = []
for s in range(1, 100):
    xs = np.asarray([s, 10, 1.0, 0.05, 0.3]).reshape((1, 5))
    opt_prc_pred.append(model_b2.predict(xs, verbose=0)[0])
    spot_prc.append(s)

plot_dual_line(
    x=spot_prc,
    y1=opt_prc,
    y2=opt_prc_pred,
    title="Step Barrier Actual vs Predicted",
    x_lab="Spot Price",
    y1_lab="Actual Option Price",
    y2_lab="Predicted Option Price",
)

## Path Dependant

Now we have completed a basic proof of concept with simple closed form, we move on to look at path dependant options with a much higher cost of compute and an even more complex payoff function.

To get going we have taken a path dependant option from an open source git repo, that is implemented as a tensor flow model that can run on a GPU. We need this as calculating millions of path dependant option values for training takes days or elapse time, even with a GPU.

This type of option allows for an early exercise, where the buyer can exit the contract before the defined maturity date. As such this is another type of feature for the neural network to learn.

So, we follow the same pattern, by creating a set of all possible features, which comes to more than 12 million points even on a fairly corse grid. The value of the option for each scenario is then calculated. This gives us a training data set, but this time it is expensive to calculate and as the payoff is more complex we need more data. This highlights the trend that will be a challenge for this approach, the more complex the option, the more data we need which is now more expensive to compute. Which is why we try to use past quotes (calculations) as we have these for free. However they may not evenly cover the feature space we need to train for

In [ ]:
#
# https://github.com/google/tf-quant-finance/blob/master/tf_quant_finance/black_scholes/crr_binomial_tree_test.py
#


def _get_payoff_fn(strikes, is_call_options):
    """Constructs the payoff functions."""
    option_signs = tf.cast(is_call_options, dtype=strikes.dtype) * 2 - 1

    def payoff(spots):
        """Computes payff for the specified options given the spot grid.

        Args:
          spots: Tensor of shape [batch_size, grid_size, 1]. The spot values at some
            time.

        Returns:
          Payoffs for exercise at the specified strikes.
        """
        return tf.nn.relu((spots - strikes) * option_signs)

    return payoff


def _get_value_modifier(is_american, payoff_fn):
    """Constructs the value modifier for american style exercise."""

    def modifier(values, spots):
        immediate_exercise_value = payoff_fn(spots)
        return tf.where(
            is_american, tf.math.maximum(immediate_exercise_value, values), values
        )

    return modifier


def option_price_binomial(
    *,
    volatilities,
    strikes,
    expiries,
    spots,
    discount_rates=None,
    dividend_rates=None,
    is_call_options=None,
    is_american=None,
    num_steps=100,
    dtype=None,
    name=None,
):
    """Computes the BS price for a batch of European or American options.

    Uses the Cox-Ross-Rubinstein version of the binomial tree method to compute
    the price of American or European options. Supports batching of the options
    and allows mixing of European and American style exercises in a batch.
    For more information about the binomial tree method and the
    Cox-Ross-Rubinstein method in particular see the references below.

    #### Example

    ```python
    # Prices 5 options with a mix of Call/Put, American/European features
    # in a single batch.
    dtype = np.float64
    spots = np.array([1.0, 2.0, 3.0, 4.0, 5.0], dtype=dtype)
    strikes = np.array([3.0, 3.0, 3.0, 3.0, 3.0], dtype=dtype)
    volatilities = np.array([0.1, 0.22, 0.32, 0.01, 0.4], dtype=dtype)
    is_call_options = np.array([True, True, False, False, False])
    is_american = np.array([False, True, True, False, True])
    discount_rates = np.array(0.035, dtype=dtype)
    dividend_rates = np.array([0.02, 0.0, 0.07, 0.01, 0.0], dtype=dtype)
    expiries = np.array(1.0, dtype=dtype)

    prices = option_price_binomial(
        volatilities=volatilities,
        strikes=strikes,
        expiries=expiries,
        spots=spots,
        discount_rates=discount_rates,
        dividend_rates=dividend_rates,
        is_call_options=is_call_options,
        is_american=is_american,
        dtype=dtype)
    # Prints [0., 0.0098847, 0.41299509, 0., 0.06046989]
    ```

    #### References

    [1] Hull, John C., Options, Futures and Other Derivatives. Pearson, 2018.
    [2] Wikipedia contributors. Binomial Options Pricing Model. Available at:
      https://en.wikipedia.org/wiki/Binomial_options_pricing_model

    Args:
      volatilities: Real `Tensor` of any shape and dtype. The volatilities to
        expiry of the options to price.
      strikes: A real `Tensor` of the same dtype and compatible shape as
        `volatilities`. The strikes of the options to be priced.
      expiries: A real `Tensor` of same dtype and compatible shape as
        `volatilities`. The expiry of each option. The units should be such that
        `expiry * volatility**2` is dimensionless.
      spots: A real `Tensor` of any shape that broadcasts to the shape of the
        `volatilities`. The current spot price of the underlying.
      discount_rates: An optional real `Tensor` of same dtype as the
        `volatilities`. The risk free discount rate. If None the rate is assumed
        to be 0.
        Default value: None, equivalent to discount rates = 0..
      dividend_rates: An optional real `Tensor` of same dtype as the
        `volatilities`. If None the rate is assumed to be 0.
        Default value: None, equivalent to discount rates = 1.
      is_call_options: A boolean `Tensor` of a shape compatible with
        `volatilities`. Indicates whether the option is a call (if True) or a put
        (if False). If not supplied, call options are assumed.
        Default value: None, equivalent to is_call_options = True.
      is_american: A boolean `Tensor` of a shape compatible with `volatilities`.
        Indicates whether the option exercise style is American (if True) or
        European (if False). If not supplied, European style exercise is assumed.
        Default value: None, equivalent to is_american = False.
      num_steps: A positive scalar int32 `Tensor`. The size of the time
        discretization to use.
        Default value: 100.
      dtype: Optional `tf.DType`. If supplied, the dtype to be used for conversion
        of any supplied non-`Tensor` arguments to `Tensor`.
        Default value: None which maps to the default dtype inferred by TensorFlow
          (float32).
      name: str. The name for the ops created by this function.
        Default value: None which is mapped to the default name `option_price`.

    Returns:
      A `Tensor` of the same shape as the inferred batch shape of the input data.
      The Black Scholes price of the options computed on a binomial tree.
    """
    with tf.name_scope(name or "crr_option_price"):
        strikes = tf.convert_to_tensor(strikes, dtype=dtype, name="strikes")
        dtype = strikes.dtype
        volatilities = tf.convert_to_tensor(
            volatilities, dtype=dtype, name="volatilities"
        )
        expiries = tf.convert_to_tensor(expiries, dtype=dtype, name="expiries")
        spots = tf.convert_to_tensor(spots, dtype=dtype, name="spots")

        if discount_rates is None:
            discount_rates = tf.zeros_like(volatilities)
        else:
            discount_rates = tf.convert_to_tensor(
                discount_rates, dtype=dtype, name="discount_rates"
            )
        if dividend_rates is None:
            dividend_rates = tf.zeros_like(volatilities)
        else:
            dividend_rates = tf.convert_to_tensor(
                dividend_rates, dtype=dtype, name="dividend_rates"
            )
        if is_call_options is None:
            is_call_options = tf.ones_like(
                volatilities, dtype=tf.bool, name="is_call_options"
            )
        else:
            is_call_options = tf.convert_to_tensor(
                is_call_options, dtype=tf.bool, name="is_call_options"
            )
        if is_american is None:
            is_american = tf.zeros_like(volatilities, dtype=tf.bool, name="is_american")
        else:
            is_american = tf.convert_to_tensor(
                is_american, dtype=tf.bool, name="is_american"
            )

        num_steps = tf.cast(num_steps, dtype=dtype)
        dt = expiries / num_steps

        # CRR choices for the up and down move multipliers
        ln_up = volatilities * tf.math.sqrt(dt)
        ln_dn = -ln_up

        # Prepares the spot grid.
        grid_idx = tf.range(num_steps + 1)
        # Stores the grid as shape [input_batch, N + 1] where N = num_steps.
        log_spot_grid_1 = tf.expand_dims(
            tf.math.log(spots) + ln_up * num_steps, axis=-1
        )
        log_spot_grid_2 = tf.expand_dims(ln_dn - ln_up, axis=-1) * grid_idx
        log_spot_grid = log_spot_grid_1 + log_spot_grid_2

        # Adding the new dimension is to ensure that batch shape is at the front.
        payoff_fn = _get_payoff_fn(
            tf.expand_dims(strikes, axis=-1), tf.expand_dims(is_call_options, axis=-1)
        )
        value_mod_fn = _get_value_modifier(
            tf.expand_dims(is_american, axis=-1), payoff_fn
        )

        # Shape [batch shape, num time steps + 1]
        values_grid = payoff_fn(tf.math.exp(log_spot_grid))

        p_up = tf.math.exp((discount_rates - dividend_rates) * dt + ln_up) - 1
        p_up /= tf.math.exp(2 * ln_up) - 1
        p_up = tf.expand_dims(p_up, axis=-1)
        p_dn = 1 - p_up
        discount_factors = tf.expand_dims(tf.math.exp(-discount_rates * dt), axis=-1)
        ln_up = tf.expand_dims(ln_up, axis=-1)

        def one_step_back(current_values, current_log_spot_grid):
            next_values = (
                current_values[..., 1:] * p_dn + current_values[..., :-1] * p_up
            )
            next_log_spot_grid = current_log_spot_grid[..., :-1] - ln_up
            next_values = value_mod_fn(next_values, tf.math.exp(next_log_spot_grid))
            return discount_factors * next_values, next_log_spot_grid

        def should_continue(current_values, current_log_spot_grid):
            del current_values, current_log_spot_grid
            return True

        batch_shape = values_grid.shape[:-1]
        pv, _ = tf.while_loop(
            should_continue,
            one_step_back,
            (values_grid, log_spot_grid),
            maximum_iterations=tf.cast(num_steps, dtype=tf.int32),
            shape_invariants=(
                tf.TensorShape(batch_shape + [None]),
                tf.TensorShape(batch_shape + [None]),
            ),
        )
        return tf.where(
            expiries > 0,
            tf.squeeze(pv, axis=-1),
            tf.where(
                is_call_options,
                tf.math.maximum(spots - strikes, 0),
                tf.math.maximum(strikes - spots, 0),
            ),
        )

In [ ]:
#
# Simple test to show the model can be tuned to price the same as the simple model
#

path_res = []
cfrm_res = []
vols = []

v = 0.05
S = 3.0
K = 3.0
T = 1.0
r = 0.035

for i in range(100):
    vols.append(v)
    path_res.append(
        np.float64(
            option_price_binomial(
                volatilities=v,
                strikes=K,
                expiries=T,
                spots=S,
                discount_rates=r,
                dividend_rates=0.0,
                is_call_options=True,
                is_american=False,
                num_steps=100,
                dtype=np.float64,
            )
        )
    )
    cfrm_res.append(black_scholes_model(S=S, K=K, T=T, r=r, v=v, o=0))
    v += 0.05

In [ ]:
plot_dual_line(
    x=vols,
    y1=path_res,
    y2=cfrm_res,
    title="Path Dependant va Closed Form - Same Option & Parameters",
    x_lab="Volatility",
    y1_lab="Path Dependant",
    y2_lab="Closed Form",
)

In [ ]:
# Differences are relativly small, as the close form is just a different way of calculating the same thing
# however, it can deal with more complex cases than the closed form.
#
np.asarray(path_res) - np.asarray(cfrm_res)

## Extend feature set

We need to extend the feature set to cover the option style, Call or Put and early exercise or not, American or European

These string features needed encoding as numerical values so they can be passed to the model.

In [ ]:
# Generate features without type and style.
vols, mats, spots, strikes, rates = parameter_scenarios(num_steps=20)
X = generate_model_scenarios(spot=spots, strike=strikes, mat=mats, rate=rates, vol=vols)

In [ ]:
# Create all combinations with option type
# Call = 0, Put = 1
X = combinations(X, [0, 1])

In [ ]:
# Create all combinations with option style
# American (early exercise) = 0, European = 1
X = combinations(X, [0, 1])

In [ ]:
# Shuffle the training data
#
print(
    f"There are now {len(X):,} training data points, for same grid size but with added features"
)
Xs = shuffle(X, random_state=42)
dfxs = pd.DataFrame(
    Xs,
    columns=[
        "spot",
        "strike",
        "maturity",
        "rate",
        "volatility",
        "call_put",
        "amer_eur",
    ],
)
dfxs.to_csv("XPathDependant.csv", encoding="ascii", index=False)

In [ ]:
!dir XPathDependant.csv

In [ ]:
def run_scenarios(X_train: np.ndarray) -> np.ndarray:
    """ Use the binomial option pricing model to price every scenrio passed in X_Train
    Args:
        X_train (np.ndarray): numpy array where each row is Spot, Strike, Time to maturity, risk free rate, volatility, is_call, is_american

    Returns:
        np.ndarray: Spot, Strike, Time to maturity, risk free rate, volatility, is_call, is_american, Price
    """
    res = []
    num_rows, num_cols = np.shape(X_train)
    S, K, T, r, v, cp, ae = np.hsplit(X_train, num_cols)  # Split by column as model can price in bulk on GPU as it is written in Tensor Flow.
    d = np.zeros((num_rows, 1)) # Div yield is always zero for our purpose.

    prices = option_price_binomial(
        volatilities=v,
        strikes=K,
        expiries=T,
        spots=S,
        discount_rates=r,
        dividend_rates=d,
        is_call_options=np.logical_not(cp),
        is_american=np.logical_not(ae),
        num_steps=200,
        dtype=np.float64)
    
    return np.hstack((X_train, prices))

In [ ]:
# Create a new CSV file for results and write a header row.

# This data is super expensive to re-create so the file create from scratch is commented out

#f = open("binomial_training.csv", "w")
#f.write(f"Spot, Strike, Maturity, rate, volatility, isPut, isEuropean, price\n")
#f.close()

In [ ]:
# Iterate over the pre-shuffled pandas X_train data in blocks, writing results out as we
# go. There are millions of training examples, so we want to make sure results are saved
# down. So, fo each block we flush, close and re-open the file in append mode.
base = 0
sz = 100000
all = len(dfxs)

while base < all:
    # Open in append.
    f = open("binomial_training.csv", "a")

    pdx = dfxs.iloc[base : base + sz]
    X_train = (pdx).to_numpy()
    results = run_scenarios(X_train)
    for r in results:
        S, K, T, r, v, cp, ae, p = r
        f.write(f"{S},{K},{T},{r},{v},{cp},{ae},{p}\n")
    f.flush()
    f.close()
    base = base + sz
    print(f"{base:,} of {all:,}")

In [ ]:
# Needs some clean post split up as it splits lines
#
def split_file(filename: str, size_in_MB: int = 10) -> None:
    """
    split the given file into smaller files of the given size

    :param filename     : The name of the file to split
    :param size_in_MB   : Size in MB of the chunked files

    :credit to: https://stackoverflow.com/users/239247/anatoly-techtonik
    """
    MAX = size_in_MB * 1024 * 1024
    BUF = 50 * 1024 * 1024 * 1024  # 50GB   - memory buffer size
    chapters = 0
    uglybuf = ""
    with open(filename, "rb") as src:
        while True:
            tgt = open(filename + ".%03d" % chapters, "wb")
            written = 0
            while written < MAX:
                if len(uglybuf) > 0:
                    tgt.write(uglybuf)
                tgt.write(src.read(min(BUF, MAX - written)))
                written += min(BUF, MAX - written)
                uglybuf = src.read(1)
                if len(uglybuf) == 0:
                    break
            tgt.close()
            if len(uglybuf) == 0:
                break
            chapters += 1
    return

In [ ]:
split_file("binomial_training_copy.csv")